# Midterm: Recommender System for Movies

**(Note: This midterm assignment will have hidden test cases)**

In this project, you will implement a recommender system for your classmates, professor and TAs based on the movie survey we have conducted. The movie preference file is at **./data/preference.csv**

## Recommender System

The objective of a Recommender System is to recommend relevant items to users, based on their preference. Recommender systems are prevalent in the digital space. For example, when you go shopping on Amazon, you notice that Amazon is recommending products on the front page before you even type anything in the search box. Similarly, when you go on YouTube, the top bar of Youtube is typically "videos recommended to you." All these features are based on recommmender systems. 

What item to recommend to which user is arguably the most important business decision in many digital platforms. For instance, YouTube cannot control the videos its users upload to it. It cannot control which videos users like to watch either. Moreoveor, since watching videos is free, YouTube cannot control the behavior of its users by changing the price of its items. It does not have inventory either since each video can be viewed as many times as possible. In this case, what could YouTube control? Or in other words, what differentiates a good video streaming service from a bad one? The answer is its recommender system. 

## Types of Recommender Systems

There are **three** types of recommender systems. 

### Popularity-based Recommendation 

The most obvious system is popularity-based recommendation. In this case, we recommend to a user the most popular items that the user has not previously consumed. In the movie setting, we will recommend the movie that most users have watched and liked. In other words, this system utilizes the "wisdom of the crowd." It usually provides good recommendations for most people. Since it is easy to implement, the popularity-based recommendation system is used as a baseline. *Note: this system is not personalized. If two consumers have not watched Movie A, and Movie A is the most popular one, both of them will be recommended Movie A, no matter how different these two consumers are.*

### Content-based Recommendation 

This recommender system leverages the data on a customer's historical actions. It first uses available data to identify a set of features that describes an item (for example, for movies, we can use the movie's director, main actor, main actress, genre, etc. to describe the movie). When a user comes in, the system will recommend the movie that is closest, in terms of these features, to the movies that the user has watched and liked. For instance, if a user likes action movies from Nolan the most, this system will recommend another action movie from Nolan that this user has not watched. *Note: we will not implement this system in this project since it requires knowledge about supervised learning. We may come back to this topic at the end of this semester.*

### Collaborative Filtering Recommendation

The last type of recommender system is called collaborative filtering. This approach uses the memory from previous users' interactions to compute users' similarities based on items they've interacted (user-based approach) or compute items' similarities based on the users that have interacted with them (item-based approach).

A typical example of this approach is User Neighbourhood-based CF, in which the top-N similar users (usually computed using Pearson correlation) for a user are selected first. The items that are liked by these users are then used to identify the best candidate to recommend to the current user. 

# 0. Read-in the preference file

The first exercise is to read in the movie preference csv file (you need to use relative path). 

You must return two things:

1. A dictionary where the key is username and the value is a vector of (-1, 0, 1) that indicates the user's preference across movies (in the order of the csv file). Note that 1 encodes a "like" and -1 encodes a "dislike". A zero means that the user has not watched that movie yet. 

2. A list of strings that contains movie names. (The order of movie names should be the same as the order in the original csv file)


**Note 1:** Your result should exactly match the results from the assert statements. This means you should pay attention to extra space, newline, etc.

**Note 2:** If there are two records with the same name, use the first record from the person.


In [1584]:
def read_in_movie_preference():
    """Read the movie data and return a preference dictionary."""
    preference = {}
    movies = []

    # Open the file
    file_location = "./data/preference.csv"
    file_reader = open(file_location, "r")

    # Extract the movie names
    first_line = file_reader.readline().strip().split(",")
    #the first line has two phrases we are not using, so we start at 2
    for index in range(2, len(first_line)):
        #append them to a list
        movies.append(first_line[index].strip())

    # Extract names and ratings 
    for line in file_reader:
        clean_line = line.strip().split(",")
        person = clean_line[1]
        rating = clean_line[2:]

        # Create a list of movie ratings for this person
        rating_list = []
        for i in range(len(movies)):
            rating_list.append(int(rating[i]))

        # Add this person's rating list to their preference dictionary
        if person in preference: 
            continue 
            
        if person not in preference:
            preference[person] = rating_list
            
    return [movies, preference]

In [1585]:
[movies, preference] = read_in_movie_preference()
assert len(movies) == 20

In [1586]:
[movies, preference] = read_in_movie_preference()
assert movies == ['The Shawshank Redemption', 'The Godfather',
                       'The Dark Knight', 'Star Wars: The Force Awakens',
                       'The Lord of the Rings: The Return of the King',
                       'Inception', 'The Matrix', 'Avengers: Infinity War',
                       'Interstellar', 'Spirited Away', 'Coco', 'The Dark Knight Rises',
                       'Braveheart', 'The Wolf of Wall Street', 'Gone Girl', 'La La Land',
                       'Shutter Island', 'Ex Machina', 'The Martian', 'Kingsman: The Secret Service']

In [1587]:
[movies, preference]=read_in_movie_preference()
assert preference["Elaine Jiang"]==[1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0]
assert preference["Songqi Fan"]==[1,0,1,1,1,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0]

In [1588]:
[movies, preference] = read_in_movie_preference()
assert preference["Jacob Scheinman"] == [1, 1, 1, 1, 1, 1, 1, 1, -1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1]
assert preference["Ziqing Ouyang"] == [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]

In [1589]:
[movies, preference] = read_in_movie_preference()
assert preference["Spencer"]==[0,1,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0]

# 1. Popularity-based Ranking
## 1.1 Compute the ranking of most popular movies

Your next task is to use the data stored in preference variable and compute the popularity scores of the movies. To compute a movie's popularity score, you should first compute the number of times people have liked movies in the entire dataset across all movies (i.e., total likes). You should then compute the number of times people have disliked movies in the entire dataset across all movies (i.e., total dislikes).

Let's assume that people have liked movies **A** times in the entire dataset and disliked movies **B** times in the entire dataset. The popularity score of a movie is then defined as **Num_of_People_Like_the_Movie - A / B * Num_of_People_Dislike_the_Movie**

(We use A/B to normalize the weights of likes and dislikes because if one type of reaction is rare, it derseves more weights. For example, if a typical movie gets on average 100 likes and no dislike, a dislike conveys a much stronger message on a movie's quality than a like).


Your function should return:
1. A dictionary where the keys are movie names and the values are correpsonding movie popularity score.
2. A list of movie names sorted descendingly by their popularity. For example, if 'The Shawshank Redemption' is the second most popular movie, the second element in the list should be 'The Shawshank Redemption'.
3. **A** and **B** as defined above.

**Note: You may want to use prior functions to help you read data inside this function**

In [1590]:
def movies_popularity_ranking():
    movie_popularity = {}
    movie_popularity_rank = []
    total_likes = 0
    total_dislikes = 0
    
    # Grab previous function 
    movies, preference = read_in_movie_preference()    
    
    # Compute A and B 
    for ratings in preference.values():
        for rating in ratings:
            if rating == 0:
                #stay the same if 0
                total_likes += 0
                total_dislikes +=0
            elif rating == 1: 
                #increase if 1
                total_likes+=1
            else:
                #decrease if -1, only possible options are -1,0,1   
                total_dislikes += 1        
    
    # Computer popularity factor, A/B 
    popularity_factor= total_likes / total_dislikes if total_dislikes != 0 else 1
    # Compute popularity scores for each movie
    for index, movie in enumerate(movies):
        num_likes = 0
        num_dislikes = 0
        for ratings in preference.values():
            if ratings[index]== 1:
                num_likes += 1
            elif ratings[index] == -1:
                num_dislikes += 1             
        popularity_score = num_likes - popularity_factor * num_dislikes
        movie_popularity[movie] = popularity_score    
        
    # Create a list of movie names sorted by popularity
    for movie in sorted(movie_popularity, key=movie_popularity.get, reverse=True):
        movie_popularity_rank.append(movie)   
        
    return movie_popularity, movie_popularity_rank, total_likes, total_dislikes

In [1591]:
movie_popularity, movie_popularity_rank, total_likes, total_dislikes = movies_popularity_ranking()
assert total_likes == 1300
assert total_dislikes == 236

In [1592]:
movie_popularity, movie_popularity_rank, total_likes, total_dislikes = movies_popularity_ranking()
assert round(movie_popularity["The Shawshank Redemption"], 2) == 66.98
assert round(movie_popularity["Avengers: Infinity War"], 2) == 14.86

In [1593]:
movie_popularity, movie_popularity_rank, total_likes, total_dislikes = movies_popularity_ranking()
assert movie_popularity_rank == ['The Shawshank Redemption',
 'Inception',
 'Kingsman: The Secret Service',
 'The Wolf of Wall Street',
 'The Matrix',
 'Coco',
 'Avengers: Infinity War',
 'The Dark Knight Rises',
 'Interstellar',
 'The Dark Knight',
 'The Martian',
 'Spirited Away',
 'The Godfather',
 'Braveheart',
 'La La Land',
 'Shutter Island',
 'Gone Girl',
 'The Lord of the Rings: The Return of the King',
 'Ex Machina',
 'Star Wars: The Force Awakens']

## 1.2 Recommendation

You now implement a polularity-based recommendation function. This function takes in a user's name. It returns a string representing the name of a movie that satisfies the following three conditions:

1. The user has not watched this movie.
2. This movie has the best popularity score (among those that are not watched by the user).
3. This movie has higher popularity score than the average of the popularity scores of the movies that this user has watched (the average is computed over all movies wateched by the user, regardless of whether they were liked by the user or not). 

If the user name does not exit, this function should return "Invalid user."

If the user has watched all movies, this function should return "Unfortunately, no new movies for you."

If the unwatched movies all have lower popularity scores than the average score of the movies watched by this user, this function should return "Unfortunately, no new movies for you."

**Note: Again, you may want to use prior functions to help you read data and rank movies inside this function**

In [1594]:
def Recommendation(name):
    recommended_movie = ""

    #grab two previous functions
    movie_popularity, movie_popularity_rank, total_likes, total_dislikes = movies_popularity_ranking()
    movies, preference = read_in_movie_preference()
    #Check if user exists
    if name not in preference:
        return "Invalid user."    
    #create two loops to find the watched and unwatched movies 
    liked_watched_movies = [movies[i] for i in range(len(movies)) if preference[name][i] ==1]
    disliked_watched_movies = [movies[i] for i in range(len(movies)) if preference[name][i]==-1]
    watched_movies = liked_watched_movies + disliked_watched_movies
    unwatched_movies = [movie for movie in movies if movie not in watched_movies]
        
    
    average_watched_score = 0
    if watched_movies: 
        average_watched_score = sum(movie_popularity[movie] for movie in watched_movies if movie in movie_popularity) / len(watched_movies) 

    #find the movie with the highest popularity score among unwatched movies  
    highest_popularity_score = -float("inf")

    for movie in unwatched_movies: 
        popularity = movie_popularity[movie]
        if popularity > highest_popularity_score:
            if popularity > average_watched_score:
                highest_popularity_score = popularity
                recommended_movie = movie
    
    if recommended_movie=="":
        return "Unfortunately, no new movies for you."
  
    return recommended_movie

In [1595]:
assert Recommendation("Yuchen") == "Unfortunately, no new movies for you."

In [1596]:
assert Recommendation("Jiaxu Rong") == 'Inception'
assert Recommendation("Nobody") == 'The Shawshank Redemption'

In [1597]:
assert Recommendation("Dennis Zhang") == 'Kingsman: The Secret Service'

In [1598]:
assert Recommendation("Test Student 2") == 'Invalid user.'

# 2.1 Cosine Similarity
Let us now use collaborative filtering to find a good recommendation.

In order to do so, we need to get the cosine similarity beween movies and users. Again, we can use the preference file we used in Section 0. The file represents each person by a preference vector that consists of (0, 1, -1). Cosine similarity in our case is the dot product of the two preference vectors divided by the product of the magnitude of the two preference vectors. In other words, if person A has preference vector A, and person B has preference vector B, their cosine similarity is equal to 

$$ \frac{A \cdot B}{||A||||B||} = \frac{\sum_i^n A_iB_i}{\sqrt{\sum_i^nA_i^2}\sqrt{\sum_i^nB_i^2}}$$

If a person has not watched any movies, then the cosine similarity between this person and any other person is defined as 0. For more information on cosine simialrity, you can read this [wiki page](https://en.wikipedia.org/wiki/Cosine_similarity)

As an example, let the following two vectors represent Naveed's and Jake's preference over 3 movies.

         Inception  Coco     The Dark Knight
    Jake     1         -1        0

    Naveed  -1          0        1

In this case, Naveed and Jake's cosine similarity is equal to

$$ \frac{1*(-1)+(-1)*0+0*(-1)}{\sqrt{1+(-1)^2}*\sqrt{(-1)^2+1}} = \frac{-1}{2} = -0.5$$

**Your task** is to write a similarity function that takes in two names and returns the Cosine similarity between these two users. If one or both names do not exist in the database, return 0. 



In [1599]:
import math 
def Similarity(name_1, name_2):
    """Given two names and preference, get the similarity 
    between two people"""
    cosine = 0
    
    # grab previous function to get preference vectors for users
    movies, preference = read_in_movie_preference()
    
    # if one or both names do not exist, return 0 
    if name_1 not in preference:
        return 0
    if name_2 not in preference: 
        return 0 
    
    #match person to their preference     
    preference_1 = preference[name_1]
    preference_2 = preference[name_2]
    
    # Calculate the cosine similarity
    
    dot_product = sum([preference_1[index] * preference_2[index] for index in range(len(movies))])
    magnitude_1 = math.sqrt(sum([preference_1[index]**2 for index in range(len(movies))]))
    magnitude_2 = math.sqrt(sum([preference_2[index]**2 for index in range(len(movies))]))
    
    # if either person has not watched any movies 
    if magnitude_1 == 0 or magnitude_2 == 0:
        return cosine
    # if not, find cosine similarity using formula 
    else:
        cosine_similarity = dot_product / (magnitude_1 * magnitude_2)
        
        return cosine_similarity 

In [1600]:
assert round(Similarity("Test Student", "Nobody"), 2) == 0.17
assert round(Similarity("Test Student", "DJZ2"), 2) == -0.27

In [1601]:
assert round(Similarity("Test Student", "Test Student 2"), 2) == 0

## 2.2 Movie Soulmate

Your next task is to find the movie soulmate of a person. In order to find a person's movie soulmate, you will compute the cosine similarity between this person and every other person in the dataset. You will then return the person who has the highest cosine similarity with the focal person. If two people have the same cosine similarity with the focal person, you can tie break by the length of names (the person with shorter name will be the soulmate). If the focal person does not exist in the database, return an empty string as the soulmate name.

Your function will return two things:
1. the name of the soulmate
2. the largest cosine similarity



In [1602]:
def Movie_Soul_Mate(name):
    """Given a name, get the player that has highest Jaccard 
    similarity with this person."""
    soulmate = ""
    cosine_similarity = -100
    
    #grab the previous function 
    movies, preference = read_in_movie_preference()
    
    #if focal person does not exist
    if name not in preference: 
        return soulmate
    # find other person with highest cosine similarity to focal person 
    for person2 in preference.keys():
        # make sure that we do not compare the focal person to himself
        if person2 != name: 
            # call the previous function that grabs the cosine similarity 
            # between the focal person and all other persons 
            cos_similarity=Similarity(name, person2)
            if cos_similarity > cosine_similarity:
                soulmate = person2
                cosine_similarity = cos_similarity 
                
            # need to check if two people have the same cosine similarity 
            elif cos_similarity == cosine_similarity:
            
            # need to pick the person with the shorter name
                if len(person2) < len (soulmate):     
                    soulmate = person2
                    
    return soulmate, cosine_similarity

In [1603]:
soulmate, cosine_similarity = Movie_Soul_Mate("Q")
assert soulmate == 'Yunong Tian'
assert round(cosine_similarity, 2) == 0.75

In [1604]:
soulmate, cosine_similarity = Movie_Soul_Mate("Test Student")
assert soulmate == 'Michael Treiber'
assert round(cosine_similarity, 2) == 0.80

In [1605]:
soulmate, cosine_similarity = Movie_Soul_Mate("Yunong Tian")
assert soulmate == 'Yuchen'
assert round(cosine_similarity, 2) == 0.81

In [1606]:
soulmate = Movie_Soul_Mate("Test Student 2")
assert soulmate == ""

## 2.3 Memory-based Collaborative Filtering Recommendation

Now after finding a person's movie soulmate, we can construct a (very preliminary) collaborative filtering recommendation. In our recommendation system, for a focal person, we first find his or her soulmate. We then find all the movies that he/she has not watched but the soulmate has watched and liked. Among all of these movies, we recommend the movie with the highest popularity score defined in Section 1.1 and 1.2 


Again, 

if the user name does not exit, this function should return "Invalid user."

If the person has watched all the movies, return "Unfortunately, no new movies for you." 

If there are no movies that are watched and liked by the soulmate but not watched by the focal person, then return the movie (or string) that should be returned in Section 1.2.


In [1607]:
def Recommendation2(name):
    # Read in movie preference data
    recommended_movie = ""
    movies, preference = read_in_movie_preference()
    movie_popularity, movie_popularity_rank, total_likes, total_dislikes = movies_popularity_ranking()
    highest_popularity_score = -float("inf")
    
    #if the username does not exist
    if name not in preference: 
        return "Invalid user."
    
    # Find user's soulmate
    soulmate, cosine_similarity = Movie_Soul_Mate(name)
    
    # Get list of movies that soulmate likes and has watched but focal person has not watched
    soulmate_likes = [movies[i] for i in range(len(movies)) if preference[soulmate][i] == 1 and preference[name][i] == 0]
    # Check if there are any movies to recommend
    if not soulmate_likes:
        # Return most popular movie
        movie_popularity, movie_popularity_rank, total_likes, total_dislikes = movies_popularity_ranking()
        return max(movie_popularity, key=movie_popularity.get)
    
    
    if len(soulmate_likes)==0:
        return "Unfortunately, no new movies for you."
    # Find movie with highest popularity score
    for movie in soulmate_likes:
        popularity = movie_popularity.get(movie, 0)
        if popularity > highest_popularity_score:
                highest_popularity_score = popularity
                recommended_movie = movie
    
    # Return recommended movie
    return recommended_movie

In [1608]:
assert Recommendation2("Test Student") == 'Inception'
assert Recommendation2("Test Student Long Name") == 'The Shawshank Redemption'

In [1609]:
assert Recommendation2("Test Student Long Name") == 'The Shawshank Redemption'

In [1610]:
assert Recommendation2("Test Student") == 'Inception'